# Waste Classification Training (Colab Ready)

Notebook này hướng dẫn toàn bộ pipeline: chuẩn bị dữ liệu, huấn luyện mô hình CNN/Transfer Learning, đánh giá và tạo demo Gradio.

## 1. Thiết lập môi trường
- Chạy trên GPU (Runtime → Change runtime type → GPU)
- Kết nối Google Drive nếu dữ liệu/model lưu trên đó.

In [ ]:
#@title Mount Google Drive (tùy chọn)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Cài đặt phụ thuộc
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install -q albumentations==1.4.7 timm==1.0.3 grad-cam==1.5.5 gradio==4.44.0 scikit-learn==1.5.2

In [ ]:
#@title Kiem tra tai nguyen Colab
import shutil
import torch

try:
    import psutil
except ImportError:
    psutil = None

print("Tinh trang tai nguyen:")
if torch.cuda.is_available():
    print(f"- GPU: {torch.cuda.get_device_name(0)}")
else:
    print("- GPU: chua bat. Vao Runtime > Change runtime type > GPU.")

total, used, free = shutil.disk_usage("/")
print(f"- O dia: {total/1e9:.1f} GB tong | {free/1e9:.1f} GB trong")

if psutil is not None:
    ram = psutil.virtual_memory()
    print(f"- RAM: {ram.total/1e9:.1f} GB tong | {ram.available/1e9:.1f} GB kha dung")
else:
    print("- RAM: cai psutil de hien thi (pip install psutil)")


## 2. Chuẩn bị dự án
- Nếu đã upload project lên GitHub, clone trực tiếp.
- Nếu làm việc trên Drive, unzip project vào `/content/TrashProject`.

In [ ]:
#@title Clone hoặc đồng bộ mã nguồn
import os, shutil, zipfile, sys

PROJECT_PATH = "/content/TrashProject"  # chỉnh sửa nếu cần
GIT_REPO_URL = "https://github.com/<your-account>/TrashProject.git"  # TODO: cập nhật

if not os.path.exists(PROJECT_PATH):
    !git clone $GIT_REPO_URL $PROJECT_PATH
else:
    print(f"Sử dụng thư mục có sẵn: {PROJECT_PATH}")

sys.path.append(PROJECT_PATH)
os.chdir(PROJECT_PATH)

## 3. Tải và tiền xử lý dữ liệu
- Có thể dùng script sẵn có để tải TrashNet về thư mục data/raw.
- Hoặc dùng dataset công khai khác/Kaggle ở các cell tiếp theo.
- Có thể thay bằng dataset tự thu thập (đưa vào thư mục data/raw).


In [ ]:
#@title Tải TrashNet từ GitHub (tùy chọn)
import os
DOWNLOAD_TRASHNET = False  #@param {type:"boolean"}
DATA_ROOT = "/content/TrashProject/data"  #@param {type:"string"}
RAW_DATA_DIR = f"{DATA_ROOT}/raw"
os.makedirs(RAW_DATA_DIR, exist_ok=True)
if DOWNLOAD_TRASHNET:
    !python scripts/download_trashnet.py --output-dir {RAW_DATA_DIR}
    print("TrashNet downloaded.")
print(f"Raw data folder: {RAW_DATA_DIR}")


In [ ]:
#@title Tải TrashNet từ Kaggle (tùy chọn)
# Yêu cầu tạo ~/.kaggle/kaggle.json trước khi chạy
DOWNLOAD_DATA = False  #@param {type:"boolean"}
DATA_ROOT = "/content/TrashProject/data"  #@param {type:"string"}

os.makedirs(DATA_ROOT, exist_ok=True)

if DOWNLOAD_DATA:
    !kaggle datasets download -d asdasdasasdas/garbage-classification
    with zipfile.ZipFile("garbage-classification.zip", "r") as zf:
        zf.extractall(DATA_ROOT)
    os.remove("garbage-classification.zip")

print(f"Data folder: {DATA_ROOT}")

In [ ]:
#@title Tải TACO từ GitHub (tùy chọn)
DOWNLOAD_TACO = False  #@param {type:"boolean"}
DATA_ROOT = "/content/TrashProject/data"  #@param {type:"string"}
RAW_DATA_DIR = f"{DATA_ROOT}/raw/taco"
os.makedirs(RAW_DATA_DIR, exist_ok=True)
if DOWNLOAD_TACO:
    !python scripts/download_taco.py --output-dir {RAW_DATA_DIR}
print(f"TACO raw data folder: {RAW_DATA_DIR}")


In [ ]:
#@title Tạo train/val/test split (70/20/10)
DATA_ROOT = globals().get('DATA_ROOT', '/content/TrashProject/data')
from pathlib import Path
import random
import shutil

raw_dir = Path(DATA_ROOT) / "raw"
train_dir = Path(DATA_ROOT) / "train"
val_dir = Path(DATA_ROOT) / "val"
test_dir = Path(DATA_ROOT) / "test"

for folder in [train_dir, val_dir, test_dir]:
    folder.mkdir(parents=True, exist_ok=True)

if raw_dir.exists():
    for class_dir in raw_dir.iterdir():
        if not class_dir.is_dir():
            continue
        images = list(class_dir.glob("*"))
        random.shuffle(images)
        n = len(images)
        n_train = int(0.7 * n)
        n_val = int(0.2 * n)
        splits = {
            train_dir / class_dir.name: images[:n_train],
            val_dir / class_dir.name: images[n_train:n_train + n_val],
            test_dir / class_dir.name: images[n_train + n_val:],
        }
        for split_dir, split_imgs in splits.items():
            split_dir.mkdir(parents=True, exist_ok=True)
            for img_path in split_imgs:
                shutil.copy(img_path, split_dir / img_path.name)
    print("Hoàn thành chia tập dữ liệu.")
else:
    print("Bỏ qua bước chia tập vì không tìm thấy data/raw.")

## 4. Cấu hình và huấn luyện
Sử dụng lớp `WasteTrainer` trong `src/training/trainer.py`.

In [ ]:
#@title Khoi tao cau hinh huan luyen
from pathlib import Path

AVAILABLE_MODELS = ["resnet18", "mobilenetv3", "efficientnetb0"]

MODEL_LIST = "resnet18"  #@param {type:"string"}
EPOCHS = 15  #@param {type:"integer"}
IMG_SIZE = 224  #@param {type:"integer"}
BATCH_SIZE = 32  #@param {type:"integer"}
NUM_WORKERS = 2  #@param {type:"integer"}
LOSS = "focal"  #@param ["cross_entropy", "focal"]
FOCAL_GAMMA = 2.0  #@param {type:"number"}
USE_BLUR = False  #@param {type:"boolean"}
USE_RANDOM_ERASING = False  #@param {type:"boolean"}
USE_MIXUP = False  #@param {type:"boolean"}
USE_CUTMIX = False  #@param {type:"boolean"}
MIXUP_ALPHA = 0.4  #@param {type:"number"}
CUTMIX_ALPHA = 1.0  #@param {type:"number"}
OPTIM = "adamw"  #@param ["adam", "adamw", "sgd"]
LR = 3e-4  #@param {type:"number"}
WEIGHT_DECAY = 1e-4  #@param {type:"number"}
SCHEDULER = "onecycle"  #@param ["onecycle", "cosine", "step", "none"]
MAX_LR = 1e-3  #@param {type:"number"}
DEVICE = "cuda"  #@param ["cuda", "cpu"]
LOG_EVERY = 20  #@param {type:"integer"}
FREEZE_BACKBONE_EPOCHS = 5  #@param {type:"integer"}
OUTPUT_DIR = "artifacts"  #@param {type:"string"}
SELECT_MODEL_METRIC = "macro_f1"  #@param ["macro_f1", "accuracy", "macro_precision", "macro_recall"]

MODEL_NAMES = [name.strip().lower() for name in MODEL_LIST.split(",") if name.strip()]
MODEL_NAMES = [name for name in MODEL_NAMES if name in AVAILABLE_MODELS]
if not MODEL_NAMES:
    MODEL_NAMES = ["resnet18"]
    print("MODEL_LIST khong hop le, su dung mac dinh resnet18.")

from src.training.dataset import DataConfig
from src.training.losses import LossConfig
from src.training.optim import OptimConfig, SchedulerConfig
from src.training.trainer import TrainConfig, WasteTrainer

DATA_ROOT = globals().get('DATA_ROOT', '/content/TrashProject/data')
data_cfg = DataConfig(
    train_dir=Path(DATA_ROOT) / "train",
    val_dir=Path(DATA_ROOT) / "val",
    test_dir=Path(DATA_ROOT) / "test",
    img_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    use_blur=USE_BLUR,
    use_random_erasing=USE_RANDOM_ERASING,
)

loss_cfg = LossConfig(name=LOSS, gamma=FOCAL_GAMMA) if LOSS == "focal" else LossConfig(name=LOSS)
optim_cfg = OptimConfig(name=OPTIM, lr=LR, weight_decay=WEIGHT_DECAY)
scheduler_cfg = None
if SCHEDULER != "none":
    scheduler_cfg = SchedulerConfig(name=SCHEDULER, max_lr=MAX_LR)

BASE_OUTPUT_DIR = Path(OUTPUT_DIR)
BASE_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

BASE_TRAIN_KWARGS = dict(
    data=data_cfg,
    loss=loss_cfg,
    optim=optim_cfg,
    scheduler=scheduler_cfg,
    epochs=EPOCHS,
    device=DEVICE,
    log_every=LOG_EVERY,
    freeze_backbone_epochs=FREEZE_BACKBONE_EPOCHS,
    use_mixup=USE_MIXUP,
    use_cutmix=USE_CUTMIX,
    mixup_alpha=MIXUP_ALPHA,
    cutmix_alpha=CUTMIX_ALPHA,
)

print(f"Models: {MODEL_NAMES}")
print(f"Epochs: {EPOCHS}, Batch size: {BATCH_SIZE}, LR: {LR}")
print(f"Loss: {LOSS}, Scheduler: {SCHEDULER}, Freeze epochs: {FREEZE_BACKBONE_EPOCHS}")
print(f"Artifacts root: {BASE_OUTPUT_DIR}")


In [ ]:
#@title Huan luyen va so sanh mo hinh
from pathlib import Path

try:
    import pandas as pd
except ImportError as exc:
    raise RuntimeError('Can install pandas trong moi truong notebook') from exc

trained_models = {}
results = []

for model_name in MODEL_NAMES:
    print(f"===== Huan luyen {model_name} =====")
    cfg = TrainConfig(
        model_name=model_name,
        output_dir=BASE_OUTPUT_DIR / model_name,
        **BASE_TRAIN_KWARGS,
    )
    trainer = WasteTrainer(cfg)
    report, cm = trainer.train()
    trained_models[model_name] = {"trainer": trainer, "report": report, "cm": cm}
    if report:
        macro = report.get("macro avg", {})
        results.append({
            "model": model_name,
            "test_accuracy": report.get("accuracy"),
            "macro_precision": macro.get("precision"),
            "macro_recall": macro.get("recall"),
            "macro_f1": macro.get("f1-score"),
        })
    else:
        results.append({
            "model": model_name,
            "test_accuracy": None,
            "macro_precision": None,
            "macro_recall": None,
            "macro_f1": None,
        })
    print()

MODEL_RESULTS_DF = None
best_trainer = None
best_report = None
best_cm = None
best_model_name = None

if results:
    MODEL_RESULTS_DF = pd.DataFrame(results).set_index("model")
    display(MODEL_RESULTS_DF)

    metric_series = MODEL_RESULTS_DF[SELECT_MODEL_METRIC]
    if metric_series.notna().any():
        best_model_name = metric_series.astype(float).idxmax()
    else:
        best_model_name = MODEL_NAMES[0]
        print("Khong co chi so hop le tren test, chon mo hinh dau tien.")
    print(f"Chon mo hinh tot nhat: {best_model_name} (theo {SELECT_MODEL_METRIC})")
    best_info = trained_models[best_model_name]
    best_trainer = best_info["trainer"]
    best_report = best_info["report"]
    best_cm = best_info["cm"]
else:
    print("Khong co ket qua nao. Hay kiem tra data/raw.")


In [ ]:
#@title Bao cao chi tiet mo hinh tot nhat
try:
    import pandas as pd
except ImportError as exc:
    raise RuntimeError('Can install pandas trong moi truong notebook') from exc

if 'best_report' in globals() and best_report:
    report_df = pd.DataFrame(best_report).T
    display(report_df)
else:
    print("Chua co ket qua nao. Hay chay cell huan luyen truoc.")


In [ ]:
#@title Hien thi ma tran nham lan
import matplotlib.pyplot as plt
import seaborn as sns
import torch

if 'best_cm' in globals() and best_cm is not None:
    cm_obj = best_cm
    cm_np = cm_obj.numpy() if isinstance(cm_obj, torch.Tensor) else cm_obj
    labels = list(best_trainer.idx_to_class.values()) if 'best_trainer' in globals() and best_trainer else []
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm_np, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()
else:
    print('Chua co ma tran nham lan. Hay chay cell huan luyen truoc.')


## 5. Demo Gradio
Tải trọng số tốt nhất và tạo UI đơn giản cho phép người dùng upload ảnh.

In [ ]:
#@title Khoi tao Gradio demo
import gradio as gr
import gradio_client.utils as gradio_utils

if 'best_trainer' not in globals() or best_trainer is None:
    raise RuntimeError('Chua co mo hinh huan luyen. Hay chay cell huan luyen truoc.')

# Work around Gradio bug when additionalProperties=False produces a bool schema
_original_json_schema_to_python_type = gradio_utils._json_schema_to_python_type

def _safe_json_schema_to_python_type(schema, defs):
    if isinstance(schema, bool):
        return 'Any'
    return _original_json_schema_to_python_type(schema, defs)

gradio_utils._json_schema_to_python_type = _safe_json_schema_to_python_type

from torchvision import transforms
from PIL import Image
import torch

model = best_trainer.model
checkpoint = torch.load(best_trainer.config.output_dir / 'best.pt', map_location=best_trainer.device)
model.load_state_dict(checkpoint['model_state'])
model.eval()

preprocess = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

class_names = [best_trainer.idx_to_class[idx] for idx in sorted(best_trainer.idx_to_class)]

def predict(image: Image.Image):
    tensor = preprocess(image).unsqueeze(0).to(best_trainer.device)
    with torch.no_grad():
        logits = model(tensor)
        probs = torch.softmax(logits, dim=1).cpu().numpy()[0]
    return {class_names[i]: float(probs[i]) for i in range(len(class_names))}

gr.Interface(fn=predict, inputs=gr.Image(type='pil'), outputs=gr.Label(num_top_classes=3)).launch(share=True)


## 6. Lưu kết quả và tải xuống
- Lưu lại mô hình, báo cáo metrics vào Drive.
- Đảm bảo cập nhật báo cáo đồ án với bảng kết quả và nhận xét.

## 7. So sánh mô hình
Sau khi chạy huấn luyện với các kiến trúc khác nhau, hãy đặt `OUTPUT_DIR` khác nhau cho mỗi lần (ví dụ `artifacts_resnet18`, `artifacts_mobilenet`).
Cell dưới đây sẽ đọc các thư mục output đó, tổng hợp Accuracy/Macro F1 trên test và best val acc để bạn so sánh nhanh.
Chỉnh sửa biến `EXPERIMENT_DIRS` theo danh sách mô hình bạn đã huấn luyện.


In [ ]:
#@title Tổng hợp kết quả các mô hình đã huấn luyện
from pathlib import Path
import torch

EXPERIMENT_DIRS = {
    "resnet18": "/content/TrashProject/artifacts_resnet18",
    "mobilenetv3": "/content/TrashProject/artifacts_mobilenetv3",
    "efficientnetb0": "/content/TrashProject/artifacts_efficientnetb0",
}  # Chỉnh sửa theo các output_dir bạn đã dùng

summary = []
for name, dir_path in EXPERIMENT_DIRS.items():
    dir_path = Path(dir_path)
    if not dir_path.exists():
        print(f"[WARN] {dir_path} không tồn tại, bỏ qua.")
        continue
    report_path = dir_path / "classification_report.pth"
    history_path = dir_path / "history.pth"
    macro_f1 = accuracy = best_val_acc = None

    if report_path.exists():
        report = torch.load(report_path)
        macro_f1 = report.get("macro avg", {}).get("f1-score")
        accuracy = report.get("accuracy")
    if history_path.exists():
        history = torch.load(history_path)
        if isinstance(history, dict) and "val_acc" in history:
            best_val_acc = max(history["val_acc"])

    summary.append({
        "model": name,
        "output_dir": str(dir_path),
        "test_accuracy": accuracy,
        "test_macro_f1": macro_f1,
        "best_val_acc": best_val_acc,
    })

if not summary:
    print("Chưa có kết quả nào. Hãy chạy huấn luyện và lưu output_dir riêng cho từng mô hình.")
else:
    header = ("Model", "Output Dir", "Test Acc", "Test Macro F1", "Best Val Acc")
    row_fmt = "{:<15} {:<40} {:>11} {:>15} {:>13}"
    print(row_fmt.format(*header))
    print('-' * 96)
    for row in summary:
        print(row_fmt.format(
            row["model"],
            row["output_dir"],
            f"{row['test_accuracy']:.3f}" if row["test_accuracy"] is not None else "n/a",
            f"{row['test_macro_f1']:.3f}" if row["test_macro_f1"] is not None else "n/a",
            f"{row['best_val_acc']:.3f}" if row["best_val_acc"] is not None else "n/a",
        ))
